# Week 02 – Foundations of Network Security  
## Networks and Systems Security

This notebook implements the **Week 02 lab** on foundational network security concepts.


In [4]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.


## Step 1 – RSA Key Generation

In [1]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

def generate_keys(private_path: str = "private_key.pem", public_path: str = "public_key.pem"):
    """Generate an RSA key pair and save them as PEM files."""
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
    )
    with open(private_path, "wb") as f:
        f.write(
            private_key.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.PKCS8,
                encryption_algorithm=serialization.NoEncryption(),
            )
        )
    public_key = private_key.public_key()
    with open(public_path, "wb") as f:
        f.write(
            public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo,
            )
        )
    print(f"✅ Keys saved: {private_path}, {public_path}")

generate_keys()


✅ Keys saved: private_key.pem, public_key.pem


## Step 2 – Receiver (Server) – Receive & Decrypt


In [2]:
import socket
import pickle
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

def run_receiver(host: str = "localhost", port: int = 65432,
                 private_path: str = "private_key.pem"):
    from cryptography.hazmat.primitives import serialization
    with open(private_path, "rb") as f:
        private_key = serialization.load_pem_private_key(f.read(), password=None)

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((host, port))
        s.listen()
        print(f"📡 Waiting for connection on {host}:{port} ...")
        conn, addr = s.accept()
        with conn:
            print(f"🔗 Connected by {addr}")
            data = b""
            while True:
                chunk = conn.recv(4096)
                if not chunk:
                    break
                data += chunk

    encrypted_key, iv, encrypted_message = pickle.loads(data)

    aes_key = private_key.decrypt(
        encrypted_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None,
        ),
    )

    cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv))
    decryptor = cipher.decryptor()
    message = decryptor.update(encrypted_message) + decryptor.finalize()
    print("🔐 Decrypted message:", message.decode())

# Run this in a separate process/terminal:
# run_receiver()


## Step 3 – Sender (Client) – Encrypt & Send


In [3]:
import os
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import pickle
import socket

def run_sender(message: bytes = b"Hello from the secure sender! This is confidential.",
               host: str = "localhost", port: int = 65432,
               public_path: str = "public_key.pem"):
    with open(public_path, "rb") as f:
        public_key = serialization.load_pem_public_key(f.read())

    aes_key = os.urandom(32)
    iv = os.urandom(16)

    cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv))
    encryptor = cipher.encryptor()
    encrypted_message = encryptor.update(message) + encryptor.finalize()

    encrypted_key = public_key.encrypt(
        aes_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None,
        ),
    )

    payload = pickle.dumps((encrypted_key, iv, encrypted_message))

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((host, port))
        s.sendall(payload)
    print("✅ Encrypted message sent!")

# Example (after starting receiver):
# run_sender()


## Group Work – Career Readiness Reflection




###  Joint Reflection (≈150 words)

During the group activity, my partner and I compared our chosen cybersecurity-related job roles and discussed why we found them appealing. Although our roles focused on slightly different areas of security, we identified several overlapping skills we both need to develop, such as cloud security knowledge, stronger Python scripting, and hands-on familiarity with industry tools (e.g., Nmap, Wireshark, Splunk). We suggested practical ways to close these gaps, including online courses, participating in CTFs, and contributing to student-led security projects.

We also highlighted each other’s strengths. For example, my partner’s prior experience in IT support and my background in programming both provide strong foundations that employers value. Overall, the discussion helped us see our progress more clearly and understand where university modules and extracurricular activities can support our career goals. It reinforced the importance of combining technical skills with communication, reflection, and continuous learning.
